# Previo
- Instalar MySQL Server: https://dev.mysql.com/downloads/windows/installer/8.0.html
- Instalar MySQL Workbench: https://dev.mysql.com/downloads/windows/installer/8.0.html
- Instalar e importar MySQL en Python (abajo):

In [ ]:
!pip install pymysql


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pymysql
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# Cargamos las variables de entorno del sistema
load_dotenv()
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
port = os.getenv("DB_PORT")

# Establecer conexión con el servidor MySQL

Proporcionar los detalles de la conexión: Nombre de usuario, contraseña, host y puerto.

In [ ]:
# Inicia la conexión con el servidor
conexion = pymysql.connect(
    host=host,
    user=user,
    password=password,
    port=int(port)
)

# Crea el objeto cursor para ejecutar comandos SQL
cursor = conexion.cursor()

# Crear la base de datos

In [ ]:
nombre_base_de_datos = 'aeropuerto_inventado1'
cursor.execute(f"CREATE DATABASE {nombre_base_de_datos};")

1

# Confirmar las transacciones realizadas y cerrar sesión

El uso del método commit() es esencial para confirmar y guardar las transacciones en la base de datos (las modificaciones no se guardan automáticamente en la base de datos en el momento en que se realizan, sino que deben confirmarse explícitamente). Sin él, las transacciones no se completarán correctamente y los cambios no se guardarán en la base de datos de manera adecuada.

In [ ]:
# Guarda cambios definitivamente: Sin commit(), las operaciones (INSERT, UPDATE, DELETE) solo son temporales y se pierden al cerrar la conexión.
conexion.commit()
conexion.close()

# Establece conexión nuevamente y crear la tabla vuelos, la cual contendrá información acerca de los vuelos del aeropuerto:

- numero_vuelo: Número de vuelo único que sirve como identificador principal (clave primaria) de la tabla.
- origen y destino: Columnas de texto (VARCHAR) para almacenar el origen y el destino del vuelo, no deben estar vacías.
- fecha_salida y fecha_llegada: Columnas de tipo DATETIME para almacenar las fechas y horas de salida y llegada del vuelo, no deben estar vacías.
- duracion_total_minutos: Un entero (INT) que almacena la duración total del vuelo en minutos (debe ser mayor a 0).
- tiene_escala: Una columna BOOLEAN que indica si el vuelo tiene o no una escala (por defecto estará a Falso).
- duracion_escala_minutos: Un entero (INT) que almacena la duración de la escala en minutos si la hay.
- compania_operadora: Una columna de texto (VARCHAR) para almacenar el nombre de la compañía que opera el vuelo.
- num_pasajeros: Un entero (INT) que representa el número de pasajeros que irán en el avión (por defecto será 0).
- num_plazas_libres: Un entero (INT) que representa el número de plazas libres en el avión.

In [ ]:
# Establecer conexión a la base de datos
conexion = pymysql.connect(
    host=host,
    user=user,
    password=password,
    port=int(port),
    database=nombre_base_de_datos
)

# Crear un cursor para ejecutar comandos SQL
cursor = conexion.cursor()

In [ ]:
# Consulta SQL para crear la tabla de vuelos
crear_tabla_vuelos = """
CREATE TABLE IF NOT EXISTS vuelos (
    numero_vuelo INT PRIMARY KEY,
    origen VARCHAR(255) NOT NULL,
    destino VARCHAR(255) NOT NULL,
    fecha_salida DATETIME NOT NULL,
    fecha_llegada DATETIME NOT NULL,
    duracion_total_minutos INT CHECK (duracion_total_minutos>0),
    tiene_escala BOOLEAN DEFAULT FALSE,
    duracion_escala_minutos INT,
    compania_operadora VARCHAR(255),
    num_pasajeros INT DEFAULT 0,
    num_plazas_libres INT
);
"""

try:
    cursor.execute(crear_tabla_vuelos)
except Exception as e:
    print(f"❌ Error al crear la tabla: {e}")
    conexion.rollback()
else:
    conexion.commit()
    print("✅ Tabla de vuelos creada exitosamente.")
finally:
    conexion.close()

✅ Tabla de vuelos creada exitosamente.


# Modifica la tabla vuelos para incluir una columna que almacene la ID del piloto y otra que indique si el vuelo es de salida o de llegada:

- Necesitamos hacer un ALTER TABLE para incluir la columna id_piloto de tipo INT y tipo_vuelo de tipo ENUM con dos opciones ('Salida' o 'Llegada').

In [ ]:
# Alteramos la tabla vuelos para añadir la columna que identifica al piloto y el tipo de vuelo
alterar_tabla_vuelos = """
ALTER TABLE vuelos
ADD COLUMN id_piloto INT NOT NULL,
ADD COLUMN tipo_vuelo ENUM('Salida', 'Llegada');
"""

conexion = pymysql.connect(
    host=host,
    user=user,
    password=password,
    port=int(port),
    database=nombre_base_de_datos)

cursor = conexion.cursor()

try:
    # Aquí se pone solo el código propenso a dar error
    cursor.execute(alterar_tabla_vuelos)
except Exception as e:
    # Lo que debe de hacer nuestro programa en caso de error
    print(f'error al alterar la tabla:{e}')
else:
    # Lo que debe de hacerse en caso de éxito
    conexion.commit()

# Crea la tabla pilotos con las siguientes columnas:

- ID del Piloto: Un identificador único para cada piloto (clave primaria autoincremental).
- Nombre: El nombre completo del piloto.
- Número de Licencia: El número de licencia de piloto del piloto (no se repite).
- Fecha de Nacimiento: La fecha de nacimiento del piloto.
- Nacionalidad: La nacionalidad del piloto.
- Experiencia: Número total de horas de vuelo (debe ser mayor a 0).
- Contacto: Información de contacto del piloto.
- Fecha de Contratación: La fecha en que el piloto fue contratado por la aerolínea.
- Salario: El salario o la tarifa que se paga al piloto por sus servicios.
- Estado de Empleo: Si el piloto está actualmente empleado, en licencia o de baja.

In [ ]:
# Consulta SQL para crear la tabla de pilotos
crear_tabla_pilotos = """
CREATE TABLE IF NOT EXISTS pilotos (
    id_piloto INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(255) NOT NULL,
    numero_licencia VARCHAR(255) UNIQUE NOT NULL,
    fecha_nacimiento DATE,
    nacionalidad VARCHAR(255),
    experiencia INT CHECK (experiencia>0),
    contacto VARCHAR(255),
    fecha_contratacion DATE,
    salario DECIMAL(10, 2),
    estado_empleo ENUM('Alta', 'Baja') NOT NULL
);
"""

# Ejecutar la consulta SQL
cursor.execute(crear_tabla_pilotos)

# Confirmar la transacción
conexion.commit()

# Altera la tabla vuelos para indicar que la columna id_piloto es una clave foránea

In [ ]:
# Consulta SQL para agregar la columna id_piloto como clave foránea
agregar_fk_id_piloto = """
ALTER TABLE vuelos
ADD FOREIGN KEY (id_piloto) REFERENCES pilotos(id_piloto);
"""

# Ejecutar la consulta SQL
cursor.execute(agregar_fk_id_piloto)

# Confirmar la transacción
conexion.commit()

# Inserta al menos tres pilotos en la tabla pilotos:

La notación %(nombre)s se utiliza en Python para definir marcadores de posición en una cadena de formato y se usa comúnmente con diccionarios para asignar valores a esos marcadores de posición. Los valores en el diccionario se asignan a los marcadores de posición en la consulta SQL, de modo que los datos se insertan correctamente en la base de datos.

In [ ]:
# Datos de los pilotos (lista de diccionarios)
pilotos_data = [
    {
        "nombre": "Juan Pérez",
        "numero_licencia": "PIL-001",
        "fecha_nacimiento": "1985-05-15",
        "nacionalidad": "Mexicana",
        "experiencia": 10,
        "contacto": "juan.perez@aerolinea.com",
        "fecha_contratacion": "2015-03-20",
        "salario": 85000.50,
        "estado_empleo": "Alta"
    },
    {
        "nombre": "María González",
        "numero_licencia": "PIL-002",
        "fecha_nacimiento": "1990-08-22",
        "nacionalidad": "Española",
        "experiencia": 7,
        "contacto": "maria.gonzalez@aerolinea.com",
        "fecha_contratacion": "2018-06-10",
        "salario": 78000.75,
        "estado_empleo": "Alta"
    },
    {
        "nombre": "Carlos Rodríguez",
        "numero_licencia": "PIL-003",
        "fecha_nacimiento": "1982-11-30",
        "nacionalidad": "Argentina",
        "experiencia": 15,
        "contacto": "carlos.rodriguez@aerolinea.com",
        "fecha_contratacion": "2010-02-05",
        "salario": 92000.00,
        "estado_empleo": "Alta"
    }
]

# Consulta SQL para INSERT con parámetros nombrados
# Los parámetros %(...)s deben coincidir con las claves del diccionario.
insertar_piloto = """
INSERT INTO pilotos (
    nombre, numero_licencia, fecha_nacimiento, nacionalidad,
    experiencia, contacto, fecha_contratacion, salario, estado_empleo
) VALUES (
    %(nombre)s, %(numero_licencia)s, %(fecha_nacimiento)s, %(nacionalidad)s,
    %(experiencia)s, %(contacto)s, %(fecha_contratacion)s, %(salario)s, %(estado_empleo)s
)
"""

# Ejecutar inserción múltiple
cursor.executemany(insertar_piloto, pilotos_data)

# Confirmar la transacción
conexion.commit()

print("Los pilotos han sido insertados exitosamente en la tabla pilotos.")

Los pilotos han sido insertados exitosamente en la tabla pilotos.


# Inserta al menos tres vuelos en la tabla vuelos (executemany) y posteriormente muestra la tabla vuelos por pantalla

In [ ]:
# Datos de los vuelos que deseas insertar
vuelo1 = {
    'numero_vuelo': 1,
    'origen': 'Ciudad A',
    'destino': 'Ciudad B',
    'fecha_salida': '2023-10-10 08:00:00',
    'fecha_llegada': '2023-10-10 10:00:00',
    'duracion_total_minutos': 120,
    'tiene_escala': False,
    'duracion_escala_minutos': None,
    'compania_operadora': 'Compañía X',
    'num_pasajeros': 100,
    'num_plazas_libres': 50,
    'id_piloto': 1  # Reemplaza con el ID de un piloto existente en la tabla pilotos
}

vuelo2 = {
    'numero_vuelo': 2,
    'origen': 'Ciudad B',
    'destino': 'Ciudad C',
    'fecha_salida': '2023-10-11 09:00:00',
    'fecha_llegada': '2023-10-11 11:30:00',
    'duracion_total_minutos': 150,
    'tiene_escala': True,
    'duracion_escala_minutos': 60,
    'compania_operadora': 'Compañía Y',
    'num_pasajeros': 120,
    'num_plazas_libres': 30,
    'id_piloto': 2
}

vuelo3 = {
    'numero_vuelo': 3,
    'origen': 'Ciudad C',
    'destino': 'Ciudad A',
    'fecha_salida': '2023-10-12 14:00:00',
    'fecha_llegada': '2023-10-12 16:30:00',
    'duracion_total_minutos': 150,
    'tiene_escala': False,
    'duracion_escala_minutos': None,
    'compania_operadora': 'Compañía Z',
    'num_pasajeros': 80,
    'num_plazas_libres': 70,
    'id_piloto': 1
}

# Lista de vuelos a insertar
vuelos_a_insertar = [vuelo1, vuelo2, vuelo3]

# Senetencia para insertar los vuelos
insertar_vuelos = """
INSERT INTO vuelos (numero_vuelo, origen, destino, fecha_salida, fecha_llegada, duracion_total_minutos,
                    tiene_escala, duracion_escala_minutos, compania_operadora, num_pasajeros, num_plazas_libres, id_piloto)
                    VALUES (%(numero_vuelo)s, %(origen)s, %(destino)s, %(fecha_salida)s, %(fecha_llegada)s, %(duracion_total_minutos)s,
        %(tiene_escala)s, %(duracion_escala_minutos)s, %(compania_operadora)s, %(num_pasajeros)s, %(num_plazas_libres)s, %(id_piloto)s)
"""

try:
    # Aquí se pone solo el código propenso a dar error
    cursor.executemany(insertar_vuelos, vuelos_a_insertar)
except Exception as e:
    # Lo que debe de hacer nuestro programa en caso de error
    print(f'error al insertar datos en la tabla:{e}')
else:
    # Lo que debe de hacerse en caso de éxito
    conexion.commit()

In [ ]:
# Consulta SQL para seleccionar todos los vuelos
seleccionar_vuelos = "SELECT * FROM vuelos"

# Ejecutar la consulta SQL
cursor.execute(seleccionar_vuelos)


# Obtener NOMBRES DE COLUMNAS (antes de fetchall)
columnas = [col[0] for col in cursor.description]

# Obtener todos los registros de la consulta
records = cursor.fetchall()  # Guardar en variable antes de iterar

# Imprimir los registros por pantalla
for registro in records:
    registro_con_nombres = dict(zip(columnas, registro))  # Combina columnas-valores
    print(registro_con_nombres)

{'numero_vuelo': 1, 'origen': 'Ciudad A', 'destino': 'Ciudad B', 'fecha_salida': datetime.datetime(2023, 10, 10, 8, 0), 'fecha_llegada': datetime.datetime(2023, 10, 10, 10, 0), 'duracion_total_minutos': 120, 'tiene_escala': 0, 'duracion_escala_minutos': None, 'compania_operadora': 'Compañía X', 'num_pasajeros': 100, 'num_plazas_libres': 50, 'id_piloto': 1, 'tipo_vuelo': None}
{'numero_vuelo': 2, 'origen': 'Ciudad B', 'destino': 'Ciudad C', 'fecha_salida': datetime.datetime(2023, 10, 11, 9, 0), 'fecha_llegada': datetime.datetime(2023, 10, 11, 11, 30), 'duracion_total_minutos': 150, 'tiene_escala': 1, 'duracion_escala_minutos': 60, 'compania_operadora': 'Compañía Y', 'num_pasajeros': 120, 'num_plazas_libres': 30, 'id_piloto': 2, 'tipo_vuelo': None}
{'numero_vuelo': 3, 'origen': 'Ciudad C', 'destino': 'Ciudad A', 'fecha_salida': datetime.datetime(2023, 10, 12, 14, 0), 'fecha_llegada': datetime.datetime(2023, 10, 12, 16, 30), 'duracion_total_minutos': 150, 'tiene_escala': 0, 'duracion_esca

# Dado que olvidamos añadir el tipo de vuelo debemos actualizar los datos para esa columna de la tabla vuelos (asumiremos que todos los vuelos son de salida)

In [ ]:
# Consulta SQL para actualizar el tipo de vuelo para cada vuelo
actualizar_tipo_de_vuelo = """
UPDATE vuelos
SET tipo_vuelo = 'Salida';
"""

# Ejecutar la consulta SQL de actualización
cursor.execute(actualizar_tipo_de_vuelo)
conexion.commit()

In [ ]:
"""
UPDATE vuelos
SET tipo_vuelo = CASE
    WHEN aeropuerto_actual = origen THEN 'Salida'
    WHEN aeropuerto_actual = destino THEN 'Llegada'
    ELSE tipo_vuelo  -- Mantiene el valor actual si no cumple condiciones
END;
"""

# Muestra la duración media de los vuelos operados por el piloto 1

In [ ]:
# Consulta SQL para calcular la duración media de los vuelos operados por el piloto 1
consulta_duracion_media = """
SELECT AVG(duracion_total_minutos)
FROM vuelos
WHERE id_piloto = 1;
"""

# Ejecutar la consulta SQL
cursor.execute(consulta_duracion_media)

# Obtener el resultado de la consulta
int(cursor.fetchone()[0])

135

# Elimina al piloto 3 de la tabla pilotos

Si fuese otro piloto incluido en la tabla vuelos como FK:

- ON DELETE CASCADE: Solo si quieres borrar automáticamente datos dependientes.
- ON DELETE SET NULL: Si es aceptable que algunos vuelos no tengan piloto asignado.
- Transacción manual: Cuando necesites control preciso sobre qué se elimina.

Ejemplo:

 ALTER TABLE vuelos
DROP FOREIGN KEY fk_piloto,
ADD CONSTRAINT fk_piloto
FOREIGN KEY (id_piloto) REFERENCES pilotos(id_piloto)
ON DELETE CASCADE;

In [ ]:
# ID del piloto que deseas eliminar
id_piloto_a_eliminar = 3

# Consulta SQL para eliminar el piloto
eliminar_piloto = """
DELETE FROM pilotos
WHERE ID_Piloto = %s;
"""

# Ejecutar la consulta SQL de eliminación
cursor.execute(eliminar_piloto, (id_piloto_a_eliminar,))

# Confirmar la transacción
conexion.commit()

# Muestra los identificadores de los vuelos junto a toda la información de sus pilotos (usa join)

In [ ]:
# Consulta SQL para obtener los identificadores de vuelos y la información de los pilotos usando JOIN
consulta_join = """
SELECT vuelos.numero_vuelo, pilotos.*
FROM vuelos
INNER JOIN pilotos ON vuelos.id_piloto = pilotos.id_piloto;
"""

# Ejecutar la consulta SQL con JOIN
cursor.execute(consulta_join)

# Obtener NOMBRES DE COLUMNAS (antes de fetchall)
columnas = [col[0] for col in cursor.description]

# Obtener los resultados de la consulta
resultados = cursor.fetchall()

# Mostrar los resultados por pantalla
for fila in resultados:
    registro_con_nombres = dict(zip(columnas, fila))
    print(registro_con_nombres)
    print("-" * 40)

{'numero_vuelo': 1, 'id_piloto': 1, 'nombre': 'Juan Pérez', 'numero_licencia': 'PIL-001', 'fecha_nacimiento': datetime.date(1985, 5, 15), 'nacionalidad': 'Mexicana', 'experiencia': 10, 'contacto': 'juan.perez@aerolinea.com', 'fecha_contratacion': datetime.date(2015, 3, 20), 'salario': Decimal('85000.50'), 'estado_empleo': 'Alta'}
----------------------------------------
{'numero_vuelo': 3, 'id_piloto': 1, 'nombre': 'Juan Pérez', 'numero_licencia': 'PIL-001', 'fecha_nacimiento': datetime.date(1985, 5, 15), 'nacionalidad': 'Mexicana', 'experiencia': 10, 'contacto': 'juan.perez@aerolinea.com', 'fecha_contratacion': datetime.date(2015, 3, 20), 'salario': Decimal('85000.50'), 'estado_empleo': 'Alta'}
----------------------------------------
{'numero_vuelo': 2, 'id_piloto': 2, 'nombre': 'María González', 'numero_licencia': 'PIL-002', 'fecha_nacimiento': datetime.date(1990, 8, 22), 'nacionalidad': 'Española', 'experiencia': 7, 'contacto': 'maria.gonzalez@aerolinea.com', 'fecha_contratacion': 

# Elimina la columna Nacionalidad de la tabla pilotos

In [ ]:
# Consulta SQL para eliminar la columna Nacionalidad de la tabla pilotos
eliminar_columna_nacionalidad = """
ALTER TABLE pilotos
DROP COLUMN nacionalidad;
"""

# Ejecutar la consulta SQL para eliminar la columna
cursor.execute(eliminar_columna_nacionalidad)

# Confirmar la transacción
conexion.commit()

# Crea un dataframe de pandas y almacénalo como una tabla dentro de la base de datos

In [ ]:
# Crear un DataFrame de pandas
data = {
    'id_piloto': [1, 2, 3],
    'nombre': ['Piloto 1', 'Piloto 2', 'Piloto 3'],
    'numero_licencia': ['Licencia1', 'Licencia2', 'Licencia3'],
    'fecha_nacimiento': ['1990-01-01', '1985-03-15', '1995-07-10'],
}

df = pd.DataFrame(data)

# Utilizar SQLAlchemy para crear un motor de conexión
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{nombre_base_de_datos}")

# Insertar el DataFrame en la base de datos como una tabla llamada 'nueva_tabla_pilotos'
df.to_sql('nueva_tabla_pilotos', con=engine, if_exists='replace', index=False)

# SQLAlchemy maneja los coomits y close automáticamente gracias a su sistema de conexiones por contexto.

print("El DataFrame ha sido almacenado como una nueva tabla en la base de datos.")

El DataFrame ha sido almacenado como una nueva tabla en la base de datos.


Hemos visto:

-CREATE

-ALTER

-DROP

-SELECT

-INSERT

-UPDATE

-DELETE

-COMMIT

-JOIN

-AVG

-to_sql